In [1]:
from rdkit import Chem
from openbabel import openbabel as ob
from rdkit import Chem, RDConfig
from rdkit.Chem import AllChem, ChemicalFeatures
from rdkit.Chem import rdPartialCharges as GMCharge
import numpy as np

In [2]:
def one_hot_encoding(x, allowable_set):
    return list(map(lambda s: x == s, allowable_set))

In [30]:
m = Chem.MolFromSmiles('C1OC1')
for atom in m.GetAtoms():
    print(atom.GetAtomicNum())

6
8
6


In [4]:
class Atom_feat:

    def atom_type_one_hot(atom):
        """One hot encoding of atom type."""
      
        allowable_set = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca',
                        'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn',
                        'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au',
                        'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb']

        return one_hot_encoding(atom.GetSymbol(), allowable_set)

    def atom_radical_electrons(atom):
        """Get number of radical electrons of an atom."""
        return [atom.GetNumRadicalElectrons()]

    def atom_explicit_valence(atom):
        """Get the explicit valence of an atom."""
        return [atom.GetExplicitValence()]

    def atom_hybridization_one_hot(atom):
        """Get the hybridization type of an atom."""
        allowable_set = [Chem.rdchem.HybridizationType.SP,
                        Chem.rdchem.HybridizationType.SP2,
                        Chem.rdchem.HybridizationType.SP3,
                        Chem.rdchem.HybridizationType.SP3D,
                        Chem.rdchem.HybridizationType.SP3D2]
        return one_hot_encoding(atom.GetHybridization(), allowable_set)

    def atom_partial_charge(atom):
        """Get Gasteiger partial charge for an atom."""
        
        gasteiger_charge = atom.GetProp('_GasteigerCharge')
        if gasteiger_charge in ['-nan', 'nan', '-inf', 'inf']:
            gasteiger_charge = 0
        return [float(gasteiger_charge)]

    def atom_chirality_type_one_hot(atom):
        """One hot encoding for the chirality type of an atom."""
        if not atom.HasProp('_CIPCode'):
            return [False, False]
        allowable_set = ['R', 'S']
        return one_hot_encoding(atom.GetProp('_CIPCode'), allowable_set)
    feat_list = [atom_type_one_hot, atom_radical_electrons,atom_explicit_valence,atom_hybridization_one_hot,atom_partial_charge,atom_chirality_type_one_hot]

In [5]:
class Bond_feat:

    def bond_type_one_hot(bond):
        """One hot encoding for the type of a bond."""
        allowable_set = [Chem.rdchem.BondType.SINGLE,
                        Chem.rdchem.BondType.DOUBLE,
                        Chem.rdchem.BondType.TRIPLE,
                        Chem.rdchem.BondType.AROMATIC]
        return one_hot_encoding(bond.GetBondType(), allowable_set)


    def bond_stereo_one_hot(bond):
        """One hot encoding for the stereo configuration of a bond."""
        allowable_set = [Chem.rdchem.BondStereo.STEREONONE,
                        Chem.rdchem.BondStereo.STEREOANY,
                        Chem.rdchem.BondStereo.STEREOZ,
                        Chem.rdchem.BondStereo.STEREOE,
                        Chem.rdchem.BondStereo.STEREOCIS,
                        Chem.rdchem.BondStereo.STEREOTRANS]
        return one_hot_encoding(bond.GetStereo(), allowable_set)

    
    feat_list = [bond_type_one_hot, bond_stereo_one_hot]




In [20]:
class Mol:

    def __init__(self, smiles=None, xyz=None, smarts=None):
        self.smiles = smiles
        self.xyz = xyz
        self.smarts = smarts
        
        ## using smiles strings 
        if smiles !=None:
            mol = Chem.MolFromSmiles(smiles)
        elif smarts!=None:
#             print('smart')
            mol = Chem.MolFromSmarts(smarts)
            mol.UpdatePropertyCache()
            
        ## smallest set of smallest rings (SSSR)
        Chem.GetSymmSSSR(mol)
        GMCharge.ComputeGasteigerCharges(mol,12)
        self.mol = mol
    
    def get_atom_feature(self):
        mol = self.mol
        num_atoms = mol.GetNumAtoms()
        Mol_atoms = np.empty(num_atoms, dtype = object)
        print('number of atoms:',num_atoms)
        mol.UpdatePropertyCache()
        self.mol = mol
        GMCharge.ComputeGasteigerCharges(mol,12)
        position = []
        for i in range(num_atoms):
            atom_features = []

            ## create atom object
            atom = mol.GetAtomWithIdx(i)
            print('this is atom:',atom)

            for feat_name in Atom_feat.feat_list:
                atom_features = np.append(atom_features,(feat_name(atom)))
            Mol_atoms[i] = atom_features
        return Mol_atoms,position


    def get_bond_feature(self):       
        mol = self.mol
        num_bonds = mol.GetNumBonds()
        adjacency = np.empty(num_bonds, dtype = object)
        Mol_bonds = np.empty(num_bonds, dtype = object)
        for i in range(num_bonds):
            bond_features = []
            bond = mol.GetBondWithIdx(i)
            u = bond.GetBeginAtomIdx()
            v = bond.GetEndAtomIdx()
            adjacency[i] = [u,v]
            for feat_name in Bond_feat.feat_list:
                bond_features = np.append(bond_features,(feat_name(bond)))
            Mol_bonds[i] = bond_features
        
        return adjacency,Mol_bonds
    

            
    
    def get_func(self):
        with open('updated_patterns_feature.txt','r') as f:
            lines = f.readlines()
               
        func_g_struct = {x.split()[0]:x.split()[1] for x in lines}
        func_g_name = {x.split()[0]:x.split()[2] for x in lines}
        bond_breaks = {x.split()[0]:x.split()[3] for x in lines}
        fg_features = {x.split()[0]:x.split()[4:10] for x in lines}

        
        mol =self.mol
        func_list = []
        for fg in func_g_struct.keys():
            func = {}
            fg_feat = []
            substruct = Chem.MolFromSmarts(fg)
#             print('substruct',substruct)
#             print('substructure',mol.GetSubstructMatch(substruct))
            hit_ats = list(mol.GetSubstructMatch(substruct))
            if len(hit_ats)>0:
                func['name'] =  func_g_name[fg]
                func['strutures'] = fg
                func['label'] = func_g_struct[fg]
                func['bond_breakNum'] =  bond_breaks[fg]
                func['position'] = hit_ats
                func['features'] = fg_features[fg]
                func_list.append(func)
        
        return func_list

In [7]:
reaction1 ='[CH2:23]1[O:24][CH2:25][CH2:26][CH2:27]1.[F:1][c:2]1[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1.[H-:22].[NH2:13][c:14]1[s:15][cH:16][cH:17][c:18]1[C:19]#[N:20].[Na+:21]>>[c:2]1([NH:13][c:14]2[s:15][cH:16][cH:17][c:18]2[C:19]#[N:20])[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1' 

In [8]:
reaction2 = 'C=Cc1cc(Br)ccc1F.CCOC(C)=O.[HH].[Pd]>>CCc1cc(Br)ccc1F'

In [55]:
reactants = reaction1.split('.')
product = reaction1.split('>>')[1]

In [56]:
print(reactants)

['[CH2:23]1[O:24][CH2:25][CH2:26][CH2:27]1', '[F:1][c:2]1[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1', '[H-:22]', '[NH2:13][c:14]1[s:15][cH:16][cH:17][c:18]1[C:19]#[N:20]', '[Na+:21]>>[c:2]1([NH:13][c:14]2[s:15][cH:16][cH:17][c:18]2[C:19]#[N:20])[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1']


In [11]:
for r in reactants:
    print(r)

[CH2:23]1[O:24][CH2:25][CH2:26][CH2:27]1
[F:1][c:2]1[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1
[H-:22]
[NH2:13][c:14]1[s:15][cH:16][cH:17][c:18]1[C:19]#[N:20]
[Na+:21]>>[c:2]1([NH:13][c:14]2[s:15][cH:16][cH:17][c:18]2[C:19]#[N:20])[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1


In [12]:
print(product)

[c:2]1([NH:13][c:14]2[s:15][cH:16][cH:17][c:18]2[C:19]#[N:20])[c:3]([N+:10](=[O:11])[O-:12])[cH:4][c:5]([F:9])[c:6]([F:8])[cH:7]1


In [54]:
m = Chem.MolFromSmarts(product)
m.GetAtomWithIdx(10).GetSymbol()

'N'

In [40]:
t_mol = Mol(smarts=product) 
frag_feat = t_mol.get_func()
for f in frag_feat:
    print(f)

{'name': 'halide', 'strutures': '[$([F,Cl,Br,I])]([#6])', 'label': '201', 'bond_breakNum': '0', 'position': [15, 14], 'features': ['-10.563', '3.219', '13.782', '0.0', '4.0', '0.0']}
{'name': 'nitrile', 'strutures': '[#6]C#N', 'label': '208', 'bond_breakNum': '0', 'position': [6, 7, 8], 'features': ['-8.873', '1.002', '9.875', '23.79', '5.0', '0.0']}
{'name': 'nitro', 'strutures': '[#6][$(N(=O)~[O;H0;-0,-1])]', 'label': '207', 'bond_breakNum': '0', 'position': [9, 10], 'features': ['-10.564', '3.219', '13.782', '0.0', '4.0', '0.0']}
{'name': 'thiophene', 'strutures': 's1cccc1', 'label': '301', 'bond_breakNum': '4', 'position': [3, 2, 6, 5, 4], 'features': ['-6.35', '-0.228', '6.121', '28.24', '5.0', '0.0']}
{'name': 'benzene', 'strutures': 'c1ccccc1', 'label': '401', 'bond_breakNum': '4', 'position': [0, 9, 13, 14, 16, 18], 'features': ['-6.718', '0.072', '6.79', '0.0', '6.0', '0.0']}


In [155]:
for fg in frag_feat:
    print(float(fg['features'][0]))

-10.563
-8.873
-10.564
-6.35
-6.718


In [50]:

    for reactant in product.split('.'):
        mol = Mol(smarts=reactant) 
        atom_position,atom_feat = mol.get_atom_feature()
        adjacency, bond_feat = mol.get_bond_feature()
        frag_feat = mol.get_func()
        print('reactants:',reactant)
        print('atom_feature:',atom_feat)
        print('adjacency',adjacency)
        print('bond',bond_feat)
        print('frag',frag_feat)

number of atoms: 19
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e4654e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b2f40>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b2f40>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e4654e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e4654e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e4654e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e46b5e80>
this is atom: <rdkit.Chem.rdchem.QueryAtom object at 0x7f89e4654e80>
this is atom: 

In [188]:
import pandas as pd

df = pd.read_csv('milligram_test_random_split.csv',header=0)
df = df.drop(columns= ['Id'])

dft = df.iloc[:100]

rea = []
pro = []
for rxn in enumerate(dft['rxn']):
    reactants = rxn[1].split(',')
    product = rxn[1].split('>>')[1]
    rea.append(reactants)
    pro.append(product)
dft.insert(loc=2,
          column='reactants',
          value=rea)
dft.insert(loc=3,
          column='product',
          value=pro)

,rxn,scaled_yield,reactants,product
0,CCOS(=O)(=O)OCC.CN(C)C=O.[H-]~[Na+].c1cnc2[nH]...,0.648,[CCOS(=O)(=O)OCC.CN(C)C=O.[H-]~[Na+].c1cnc2[nH...,CCn1cnc2ncccc21
1,CN(C)C=O.ClCc1ccccc1.O=C([O-])[O-]~[K+]~[K+].c...,0.415,[CN(C)C=O.ClCc1ccccc1.O=C([O-])[O-]~[K+]~[K+]....,c1ccc(Cn2c(CCc3nc4ccccc4n3Cc3ccccc3)nc3ccccc32...
2,CC(C)=O.CCN(CC)CC.NCC(=O)N1C[C@H](NC(=O)c2cccc...,0.182,[CC(C)=O.CCN(CC)CC.NCC(=O)N1C[C@H](NC(=O)c2ccc...,O=C(N[C@@H]1C[C@@H](C(=O)O)N(C(=O)CNC(=O)C(F)(...
3,CC(=O)OC(C)=O.NCC(=O)N1C[C@H](NC(=O)c2ccccc2)C...,0.286,[CC(=O)OC(C)=O.NCC(=O)N1C[C@H](NC(=O)c2ccccc2)...,O=CNCC(=O)N1C[C@H](NC(=O)c2ccccc2)C[C@H]1C(=O)O
4,CC(=O)Oc1ccc(-c2cnc(N)c(Cc3ccccc3)n2)cc1.O=C([...,0.276,[CC(=O)Oc1ccc(-c2cnc(N)c(Cc3ccccc3)n2)cc1.O=C(...,CC(=O)Oc1ccc(-c2cnc(NS(=O)(=O)Cc3ccccc3)c(Cc3c...
...,...,...,...,...
95,C1CCOC1.CCN(CC)CC.CS(=O)(=O)Cl.Nc1ncnn2c(C3CCN...,0.293,[C1CCOC1.CCN(CC)CC.CS(=O)(=O)Cl.Nc1ncnn2c(C3CC...,CS(=O)(=O)N1CCC(c2cc(-c3ccc4cn(Cc5ccccc5)nc4c3...
96,CC(C)(C)[Si](C)(C)OCCBr.Nc1ncnn2c(C3CCNC3)cc(-...,0.520,[CC(C)(C)[Si](C)(C)OCCBr.Nc1ncnn2c(C3CCNC3)cc(...,CC(C)(C)[Si](C)(C)OCCN1CCC(c2cc(-c3ccc4cn(Cc5c...
97,C1CCNC1.CCN(C(C)C)C(C)C.CN(C)C=O.CS(=O)(=O)OCC...,0.058,[C1CCNC1.CCN(C(C)C)C(C)C.CN(C)C=O.CS(=O)(=O)OC...,Nc1ncnn2c(CC3CC3CN3CCCC3)cc(-c3ccc4cn(Cc5ccccc...
98,C1CCOC1.C1COCCO1.CC(C)(C)OC(=O)N1CCC(C(=O)c2cc...,0.778,[C1CCOC1.C1COCCO1.CC(C)(C)OC(=O)N1CCC(C(=O)c2c...,Nc1ncnn2c(C(=O)C3CCNC3)cc(-c3ccc4cn(Cc5ccccc5)...


In [201]:
a = dft['reactants'][0]
a[0].split('.')

['CCOS(=O)(=O)OCC',
 'CN(C)C=O',
 '[H-]~[Na+]',
 'c1cnc2[nH]cnc2c1>>CCn1cnc2ncccc21']